# Struttura dei test

Di seguito un'analisi sulle performance del modulo implementato (PPGA). Andremo
quindi a confrontare:

- Versione sequenziale e versione parallela di PPGA.
- Confronto tra PPGA e DEAP (sequenziale e parallelo).
- Considerazioni finali sul problema specifico del paper per un corretto
  utilizzo della versione parallela ed evitare comportamenti inattesi.

I benchmark sono stati effettuati su una macchina con doppio processore AMD
EPYC 7313, ciascuno dei quali a 16 core con frequenza di clock massima a 3.7
GHz. Il problema di riferimento è quello di explainability in cui è stato
eseguito l'algoritmo genetico su un solo punto del dataset (32 features) e su
una sola classe target. I parametri presi in considerazione sono:

- **Modello di ML**: RandomForestClassifier, SVC e MLPClassifier.
- **Numero di individui sintetici**: 1000, 2000, 4000, 8000, 16000.
- **Numero di worker**: 1, 2, 4, 8, 16, 32. Le esecuzioni con 1 worker
  corrispondono in realtà alla versione sequenziale dell'algoritmo.

Sono state esplorate tutte le possibili combinazioni di tali parametri e, per
ciascuna di esse sono stati eseguiti 10 test. I risultati dei 10 test sono
stati poi aggregati prendendo media e deviazione standard dei tempi registrati.

Sia per PPGA che per DEAP sono stati misurati sia il tempo d'esecuzione totale,
sia il tempo specifico speso in parallelo, che, nel caso di PPGA comprende le
fasi di crossover, mutazione e valutazione, mentre per DEAP comprende solo la
fase di valutazione.

# Analisi PPGA

Per prima cosa si vogliono paragonare le prestazioni offerte dalla versione
parallela dell'algoritmo rispetto a alla sua versione sequenziale.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

ppga_df = pd.read_csv("../datasets/ppga_benchmark_32.csv")
ppga_df

## Tempo d'esecuzione

Iniziamo con un semplice grafico che riporta l'andamento del tempo d'esecuzione
in funzione del diverso numero di worker e su diverse dimensioni dell'input.


In [ ]:
def plot_time(df: pd.DataFrame):
    models = df["classifier"].unique()
    workers = df["workers"].unique()
    population_sizes = df["population_size"].unique()

    fig, axes = plt.subplots(3, 2, figsize=(16, 16))

    for i, model in enumerate(models):
        for ps in population_sizes:
            mask = (df["classifier"] == model) & (df["population_size"] == ps)
            axes[i][0].set_title(f"{model} total execution time")
            axes[i][0].set_xlabel("Workers")
            axes[i][0].set_ylabel("Time (seconds)")
            axes[i][0].errorbar(
                workers,
                df[mask]["time"].values,
                df[mask]["time_std"],
                label=f"Population size: {ps}",
            )

            axes[i][0].set_xscale("log", base=2)
            axes[i][0].set_xticks(workers, labels=[int(2**w) for w in np.log2(workers)])
            axes[i][0].grid()
            axes[i][0].legend()

            axes[i][1].set_title(f"{model} partial execution time")
            axes[i][1].set_xlabel("Workers")
            axes[i][1].set_ylabel("Time (seconds)")
            axes[i][1].errorbar(
                workers,
                df[mask]["ptime"].values,
                df[mask]["ptime_std"],
                label=f"Population size: {ps}",
            )

            axes[i][1].set_xscale("log", base=2)
            axes[i][1].set_xticks(workers, labels=[int(2**w) for w in np.log2(workers)])
            axes[i][1].grid()
            axes[i][1].legend()

    plt.tight_layout()
    plt.show()


plot_time(ppga_df)

## Speed up

Procediamo con il calcolare lo _speed up_ ottenuto per ogni modello e per ogni
numero di individui sintetici. Anche qui viene calcolato sia lo speed up
ottenuto sull'esecuzione dell'intero algoritmo, sia quello ottenuto solo sulla
parte parallelizzata.


In [ ]:
reference_times = ppga_df[ppga_df["workers"] == 1].set_index(
    ["classifier", "population_size"]
)["time"]

ppga_df["speed_up"] = ppga_df.apply(
    lambda row: reference_times.loc[(row["classifier"], row["population_size"])]
    / row["time"],
    axis=1,
)

reference_ptimes = ppga_df[ppga_df["workers"] == 1].set_index(
    ["classifier", "population_size"]
)["ptime"]

ppga_df["pure_speed_up"] = ppga_df.apply(
    lambda row: reference_ptimes.loc[(row["classifier"], row["population_size"])]
    / row["ptime"],
    axis=1,
)

ppga_df

Di seguito vari grafici per vedere l'andamento dello speed up in relazione
al numero di individui sintetici generati.


In [ ]:
def plot_speed_up(df: pd.DataFrame):
    models = df["classifier"].unique()
    workers = df["workers"].unique()
    population_sizes = df["population_size"].unique()

    fig, axes = plt.subplots(3, 2, figsize=(16, 16))

    for i, model in enumerate(models):
        for ps in population_sizes:
            mask = (df["classifier"] == model) & (df["population_size"] == ps)
            axes[i][0].set_title(f"{model} speed up")
            axes[i][0].set_xlabel("Workers")
            axes[i][0].set_ylabel("Speed up")
            axes[i][0].errorbar(
                workers,
                df[mask]["speed_up"].values,
                label=f"Population size: {ps}",
            )

            axes[i][0].set_xscale("log", base=2)
            axes[i][0].set_xticks(workers, labels=[int(2**w) for w in np.log2(workers)])
            axes[i][0].grid()
            axes[i][0].legend()

            axes[i][1].set_title(f"{model} pure speed up")
            axes[i][1].set_xlabel("Workers")
            axes[i][1].set_ylabel("Speed up")
            axes[i][1].errorbar(
                workers,
                df[mask]["pure_speed_up"].values,
                label=f"Population size: {ps}",
            )

            axes[i][1].set_xscale("log", base=2)
            axes[i][1].set_xticks(workers, labels=[int(2**w) for w in np.log2(workers)])
            axes[i][1].grid()
            axes[i][1].legend()

    plt.tight_layout()
    plt.show()


plot_speed_up(ppga_df)

Come è facile notare, il Random Forest, che è anche il più lento, è quello che
trae più beneficio da una parallelizzazione più pesante. Gli altri modelli
sono molto più performanti in fase di predizione ed è quindi più difficile
ottenere un valore di speed up vicino al numero di worker utilizzati.

## Stabilità

Come ultima analisi vorrei valutare la stabilità delle performance
dell'algoritmo stesso andando a calcolare il coefficiente di variazione
del tempo impiegato. Andiamo quindi a calcolare

$$\frac{\sigma}{\mu}$$

dove $\mu$ e $\sigma$ sono media e deviazione standard dei tempi d'esecuzione
ottenuti da 10 prove ripetute dello stesso test. Questo dovrebbe darci indicare
approssimativamente quanto sia facile prevedere il tempo d'esecuzione
dell'algoritmo.


In [ ]:
ppga_df["var_coeff"] = ppga_df["time_std"] / ppga_df["time"]
ppga_df

In questo modo è possibile vedere la stabilità dell'algoritmo al variare di
parametri come numero di worker e dimensioni della popolazione. Come soglia
per separare i test più stabili dagli altri ho scelto un valore del 10% per
il coefficiente di variabilità.


In [ ]:
def plot_var_coeff(df: pd.DataFrame):
    models = df["classifier"].unique()
    workers = np.log2(np.array(df["workers"].unique()))
    population_sizes = df["population_size"].unique()

    fig, axes = plt.subplots(3, 1, figsize=(12, 18))

    for model, ax in zip(models, axes):
        ax.plot([-0.5, 5.5], [0.1, 0.1], c="r", label="low variability threshold")
        for i, ps in enumerate(population_sizes):
            mask = (df["classifier"] == model) & (df["population_size"] == ps)
            ax.set_title(f"{model} execution time variability")
            ax.set_xlabel("Workers")
            ax.set_ylabel("Variability coefficient")
            ax.bar(
                workers + i * 0.105,
                df[mask]["var_coeff"].values,
                width=0.1,
                label=f"Population size: {ps}",
            )

            ax.set_xticks(workers, labels=[int(2**w) for w in workers])
            ax.grid()
            ax.legend()

    plt.tight_layout()
    plt.show()


plot_var_coeff(ppga_df)

Dai test effettuati sembra che al variare del numero di worker e della
dimensione della popolazione il coefficiente di variabilità sia sempre sotto
il 10%.

# Analisi DEAP con multiprocessing

Il modulo DEAP offre due metodi per parallelizzare l'algoritmo genetico, il
primo tramite il modulo `multiprocessing` e il secondo tramite `scoop`. Di
seguito un'analisi della versione tramite `multiprocessing`. Le analisi sono
perfettamente analoghe alle precedenti.


In [ ]:
deap_df = pd.read_csv("../datasets/deap_mp_MLPClassifier_32.csv")
deap_df

## Tempo d'esecuzione

Come prima, alcuni plot rappresentati dell'andamento del tempo d'esecuzione in
funzione del diverso numero di worker e su diverse dimensioni dell'input.


In [ ]:
plot_time(deap_df)

## Speed up

Come prima andiamo ad analizzare lo speed up ottenuto.


In [ ]:
reference_times = deap_df[deap_df["workers"] == 1].set_index(
    ["classifier", "population_size"]
)["time"]

deap_df["speed_up"] = deap_df.apply(
    lambda row: reference_times.loc[(row["classifier"], row["population_size"])]
    / row["time"],
    axis=1,
)

# reference_ptimes = deap_df[deap_df["workers"] == 1].set_index(
#     ["classifier", "population_size"]
# )["ptime"]

# deap_df["pure_speed_up"] = deap_df.apply(
#     lambda row: reference_ptimes.loc[(row["classifier"], row["population_size"])]
#     / row["ptime"],
#     axis=1,
# )

deap_df

Notiamo, sia dalla tabella che dal grafico di seguito, che lo speed up tramite
il modulo multiprocessing è generalmente migliore rispetto all'altro algoritmo.
Questo probabilmente dovuto a tecniche di bilanciamento del carico e di
comunicazione più avanzate. Il confronto vero e proprio sarà fatto di seguito.


In [ ]:
plot_speed_up(deap_df)

## Stabilità

Come in precedenza andiamo a calcolare il coefficiente di variabilità
dell'algoritmo.


In [ ]:
deap_df["var_coeff"] = deap_df["time_std"] / deap_df["time"]
deap_df

A differenza del caso precedente è possibile, anche solo visivamente, notare
una maggiore variabilità nei tempi di esecuzione, soprattutto nel caso della
SVM e del MultiLayer Perceptron.


In [ ]:
plot_var_coeff(deap_df)

# Confronto con DEAP

Passiamo a confrontare le prestazioni con la libreria DEAP. Procederemo
confrontando tre parametri:

- Tempo d'esecuzione
- Speed up
- Coefficiente di speed up

L'ultimo è un parametro che ci permette di paragonare lo speed up in modo
proporzionale tra i due algoritmi che possono aver ottenuto tempi d'esecuzione
differenti.

## Tempo d'esecuzione

Il primo confronto è semplicemente sul tempo d'esecuzione puro dei due
algoritmi:


In [ ]:
time_cmp = pd.DataFrame(
    {
        "classifier": ppga_df["classifier"].values,
        "population_size": ppga_df["population_size"].values,
        "workers": ppga_df["workers"].values,
        "ppga": ppga_df["time"].values,
        "deap": deap_df["time"].values,
    }
)

time_cmp

Come possiamo vedere dai grafici di seguito sembra che DEAP sia leggermente
più lento.


In [ ]:
def plot_time_diff(df):
    models = df["classifier"].unique()
    workers = np.log2(df["workers"].unique())
    population_sizes = df["population_size"].unique()

    fig, axes = plt.subplots(len(population_sizes), len(models), figsize=(12, 18))

    for i, ps in enumerate(population_sizes):
        for j, model in enumerate(models):
            mask = (df["classifier"] == model) & (df["population_size"] == ps)
            axes[i][j].set_title(f"{model} execution time\nPopulation size {ps}")
            axes[i][j].set_xlabel("Workers")
            axes[i][j].set_ylabel("Time (seconds)")
            axes[i][j].bar(
                workers - 0.11, df[mask]["ppga"].values, width=0.2, label="ppga"
            )
            axes[i][j].bar(
                workers + 0.11, df[mask]["deap"].values, width=0.2, label="deap"
            )

            axes[i][j].set_xticks(workers, labels=[int(2**i) for i in workers])
            axes[i][j].legend()
            axes[i][j].grid()

    plt.tight_layout()
    plt.show()


plot_time_diff(time_cmp)

Di seguito un confronto tra il numero di volte in cui `ppga` ha offerto
performance migliori rispetto a DEAP.


In [ ]:
ppga_better_time = time_cmp[time_cmp["ppga"] < time_cmp["deap"]]
print(
    f"ppga better than deap {len(ppga_better_time) / len(time_cmp) * 100.0:.2f}% of the times"
)

Andiamo a vedere i valori minimi, medi e massimi dell'aumento di performance.


In [ ]:
improvement = (
    (ppga_better_time["deap"] - ppga_better_time["ppga"])
    / ppga_better_time["deap"]
    * 100.0
)
print(f"minimum improvement: {improvement.min():.2f}%")
print(f"mean improvement: {improvement.mean():.2f}%")
print(f"max improvement: {improvement.max():.2f}%")

Come vediamo `ppga` è tendenzialmente più veloce, mediamente del 20%.

Ripetiamo la stessa analisi ma considerando solo i casi in cui DEAP ha fornito
le performance migliori.


In [ ]:
deap_better_time = time_cmp[time_cmp["deap"] < time_cmp["ppga"]]
print(
    f"deap better than ppga {len(deap_better_time) / len(time_cmp) * 100.0:.2f}% of the times"
)

improvement = (
    (deap_better_time["ppga"] - deap_better_time["deap"])
    / deap_better_time["ppga"]
    * 100.0
)
print(f"minimum improvement: {improvement.min():.2f}%")
print(f"mean improvement: {improvement.mean():.2f}%")
print(f"max improvement: {improvement.max():.2f}%")

Non sembrano esserci grandi differenze per quanto riguarda i valori minimi e
medi, ma è stato registrato almeno un caso in cui DEAP ha fornito performance
migliori del 50% rispetto a PPGA.

## Speed up

Passiamo ora al confronto dello speed up tra i due algoritmi. Le analisi saranno
analoghe a quelle fatte per il tempo d'esecuzione.


In [ ]:
speedup_cmp = pd.DataFrame(
    {
        "classifier": ppga_df["classifier"].values,
        "population_size": ppga_df["population_size"].values,
        "workers": ppga_df["workers"].values,
        "ppga": ppga_df["speed_up"].values,
        "deap": deap_df["speed_up"].values,
    }
)

speedup_cmp = speedup_cmp[speedup_cmp["workers"] != 1]
speedup_cmp

Stavolta sembra che DEAP abbia valori di speed up migliori, come si può notare
dai grafici riportati di seguito.


In [ ]:
def plot_time_diff(df):
    models = df["classifier"].unique()
    workers = np.log2(df["workers"].unique())
    population_sizes = df["population_size"].unique()

    fig, axes = plt.subplots(len(population_sizes), len(models), figsize=(12, 18))

    for i, ps in enumerate(population_sizes):
        for j, model in enumerate(models):
            mask = (df["classifier"] == model) & (df["population_size"] == ps)
            axes[i][j].set_title(f"{model} speed up\nPopulation size {ps}")
            axes[i][j].set_xlabel("Workers")
            axes[i][j].set_ylabel("Speed up")
            axes[i][j].bar(
                workers - 0.11, df[mask]["ppga"].values, width=0.2, label="ppga"
            )
            axes[i][j].bar(
                workers + 0.11, df[mask]["deap"].values, width=0.2, label="deap"
            )

            axes[i][j].set_xticks(workers, labels=[int(2**i) for i in workers])
            axes[i][j].legend()
            axes[i][j].grid()

    plt.tight_layout()
    plt.show()


plot_time_diff(speedup_cmp)

Andiamo però a vedere quante volte DEAP è stato migliore di PPGA come abbiamo
fatto in precedenza.


In [ ]:
ppga_better_speedup = speedup_cmp[speedup_cmp["ppga"] > speedup_cmp["deap"]]
print(
    f"ppga better than deap {len(ppga_better_speedup) / len(speedup_cmp) * 100.0:.2f}% of the times"
)

improvement = (
    (ppga_better_speedup["ppga"] - ppga_better_speedup["deap"])
    / ppga_better_speedup["deap"]
    * 100.0
)
print(f"minimum improvement: {improvement.min():.2f}%")
print(f"mean improvement: {improvement.mean():.2f}%")
print(f"max improvement: {improvement.max():.2f}%")

In [ ]:
deap_better_speedup = speedup_cmp[speedup_cmp["deap"] > speedup_cmp["ppga"]]
print(
    f"deap better than ppga {len(deap_better_speedup) / len(speedup_cmp) * 100.0:.2f}% of the times"
)

improvement = (
    (deap_better_speedup["deap"] - deap_better_speedup["ppga"])
    / deap_better_speedup["ppga"]
    * 100.0
)
print(f"minimum improvement: {improvement.min():.2f}%")
print(f"mean improvement: {improvement.mean():.2f}%")
print(f"max improvement: {improvement.max():.2f}%")

In questo caso DEAP risulta avere i migliori valori di speed la maggior parte
delle volte.

## Coefficiente di speed up

Confrontare il tempo d'esecuzione può essere un buon indice per paragonare le
performance di un algoritmo, quando però andiamo a paragonare i valori di speed
up non otteniamo una vera e propria misura di quanto un modello di calcolo
parallelo sia migliore rispetto ad un altro. Questo è anche dovuto al fatto
che il tempo d'esecuzione, a parità di worker impiegati, potrebbe variare.
Diventa quindi necessario ricavare una misura che sia proporzionale al tempo
d'esecuzione. Di seguito viene calcolato un "coefficiente di speed up"

$$\frac{s}{t}$$

dove $s$ è lo speed up ottenuto e $t$ il tempo totale d'esecuzione.


In [ ]:
speedup_coeff_cmp = pd.DataFrame(
    {
        "classifier": ppga_df["classifier"].values,
        "population_size": ppga_df["population_size"].values,
        "workers": ppga_df["workers"].values,
        "ppga": (ppga_df["speed_up"] / ppga_df["time"]).values,
        "deap": (deap_df["speed_up"] / deap_df["time"]).values,
    }
)

speedup_coeff_cmp = speedup_coeff_cmp[speedup_coeff_cmp["workers"] != 1]
speedup_coeff_cmp

Dalla tabella e dai grafici riportati di seguito possiamo notare come PPGA
offra, nella maggior parte dei casi, un coefficiente migliore.

Dobbiamo però considerare anche l'andamento di tale coefficiente caso per caso.
Se rimane stabile o tende a crescere con il crescere del numero di worker,
potrebbe essere un buon indice per valutare la scalabilità dell'algoritmo
su un numero ancora più alto di worker.


In [ ]:
def plot_time_diff(df):
    models = df["classifier"].unique()
    workers = np.log2(df["workers"].unique())
    population_sizes = df["population_size"].unique()

    fig, axes = plt.subplots(len(population_sizes), len(models), figsize=(12, 18))

    for i, ps in enumerate(population_sizes):
        for j, model in enumerate(models):
            mask = (df["classifier"] == model) & (df["population_size"] == ps)
            axes[i][j].set_title(f"{model} speed up\nPopulation size {ps}")
            axes[i][j].set_xlabel("Workers")
            axes[i][j].set_ylabel("Speed up coefficient")
            axes[i][j].bar(
                workers - 0.11, df[mask]["ppga"].values, width=0.2, label="ppga"
            )
            axes[i][j].bar(
                workers + 0.11, df[mask]["deap"].values, width=0.2, label="deap"
            )

            axes[i][j].set_xticks(workers, labels=[int(2**i) for i in workers])
            axes[i][j].legend()
            axes[i][j].grid()

    plt.tight_layout()
    plt.show()


plot_time_diff(speedup_coeff_cmp)

Come già detto, il coefficiente vuole essere un indice di quanto l'algoritmo
scala bene con il crescere del numero di worker. Come possiamo vedere, una
casistica in cui il carico dei singoli worker tende ad essere leggero, un
approccio in cui c'è un bilanciamento automatico e dinamico del carico di lavoro
risulta essere vincente sia in termini di tempo d'esecuzione, sia in termini di
utilizzo delle risorse fornite (in questo caso i core multipli della macchina).

Quando però il workload si fa più intenso, sembra essere più vantaggioso
partizionare subito l'input e lasciare lavorare i worker senza ulteriori
overhead di sincronizzazione e comunicazione.


In [ ]:
ppga_better_coeff = speedup_coeff_cmp[
    speedup_coeff_cmp["ppga"] > speedup_coeff_cmp["deap"]
]

print(
    f"deap better than ppga {len(ppga_better_coeff) / len(speedup_coeff_cmp) * 100.0:.2f}% of the times"
)

improvement = (
    (ppga_better_coeff["ppga"] - ppga_better_coeff["deap"])
    / ppga_better_coeff["deap"]
    * 100.0
)
print(f"minimum improvement: {improvement.min():.2f}%")
print(f"mean improvement: {improvement.mean():.2f}%")
print(f"max improvement: {improvement.max():.2f}%")

In [ ]:
deap_better_coeff = speedup_coeff_cmp[
    speedup_coeff_cmp["deap"] > speedup_coeff_cmp["ppga"]
]

print(
    f"deap better than ppga {len(deap_better_coeff) / len(speedup_coeff_cmp) * 100.0:.2f}% of the times"
)

improvement = (
    (deap_better_coeff["deap"] - deap_better_coeff["ppga"])
    / deap_better_coeff["ppga"]
    * 100.0
)
print(f"minimum improvement: {improvement.min():.2f}%")
print(f"mean improvement: {improvement.mean():.2f}%")
print(f"max improvement: {improvement.max():.2f}%")

# Conclusioni

In quanto a tempo d'esecuzione sembra prevalere PPGA con un buon 80% dei test
eseguiti con un tempo inferiore a DEAP. Di questo 80% c'è un miglioramento
medio del 20% e un miglioramento massimo del 40%.

Sui valori di speed up delle versioni parallele DEAP riesce in diversi casi a
sfruttare meglio le risorse computazionali a disposizione. La differenza tende
però ad appianarsi quando il carico di lavoro di ogni worker si fa sempre più
grande.

Per l'ultimo coefficiente valgono più o meno le stesse considerazioni fatte
sullo speed up. Come si nota dai grafici, i casi in cui il coefficiente è più
alto per PPGA sono quelli con pochi worker, che dunque saranno più carichi
di lavoro. Qui diventa determinate il tempo d'esecuzione, tendenzialmente
inferiore, di PPGA.

Se però entrano in gioco un alto numero di worker e workload eterogenei, la
miglior scelta ricade sul `multiprocessing.Pool` utilizzato da DEAP.

## Considerazioni parallelismo

Prima di concludere vorrei evidenziare alcune possibili criticità quando si
lavora con la versione parallela di PPGA, prendendo sempre come riferimento il
solito problema. A differenza di DEAP, che parallelizza solo la fase di
valutazione, PPGA parallelizza anche le fasi di crossover, mutazione.

Durante la fase di test sono emersi alcuni problemi, anche se in un sottoinsieme
ridotto di casistiche, che hanno portato alla luce alcuni fattori a cui fare
attenzione.

Dato che la genereazione della popolazione iniziale avviene per copia, la
diversità genetica iniziale è pari a zero. Per un algoritmo genetico, questo
è in generale un problema dato che sfrutta meccanismi come il crossover per
ricombinare i cromosomi tra individui diversi.

Nel nostro caso partiamo però da una situazione in cui tutti gli individui sono
cloni e utilizziamo metodi di crossover come il crossover singolo o doppio, il
quale ricombina le feature tra due cromosomi senza però alterare il loro ordine.

Ne segue che l'applicazione del crossover tra due cloni risulterà nella
generazione di altri due cloni identici tra di loro e ai loro genitori. Per le
prime iterazioni ci si affida quindi all'operatore di mutazione come unico
meccanismo per introdurre diversità genetica.

Il problema è che la mutazione ha generalmente una probabilità bassa di essere
applicata (in genere 20/30%). Se aggiungiamo poi che ogni feature ha una
probabilità indipendente di essere modificata, la probabilità che un individuo
venga effettivamente modificato cala ulteriormente.

Aggiungiamo infine che quando partizioniamo la popolazione tra molti worker si
potrebbe andare incontro ad un problema nel caso in cui il rapporto

$$\frac{N}{W}$$

dove $N$ è il numero di individui e $W$ è il numero di worker, è troppo basso.
Si rischia che i worker mutino pochi individui o nessuno, lasciando la
diversità genetica bassa per diverse iterazioni.

Per evitare che succeda c'è bisogno che la probabilità indipendente di
modificare ogni feature faccia sì che mediamente, almeno una feature venga
modificata. Per esempio ponendola a $1 / L$ dove $L$ è la lunghezza del
cromosoma.

In secondo luogo è necessario che il numero di individui inviato ad ogni worker
sia abbastanza grande da "garantire" un certo numero di individui mutati.

Ultima considerazione che potrebbe essere banale è impostare manualmente il
_seed_ dei generatori pseudocasuali di ogni worker, altrimenti inizializzato
con l'orologio ad una risoluzione dell'ordine dei secondi.

Ne seguirebbe che ogni worker è inizializzato quasi sicuramente con lo stesso
seed e quindi muta gli stessi individui allo stesso modo degli altri worker.
Andiamo quindi a creare $W$ sottopopolazioni tutte identiche tra di loro.

## Ulteriore strategia di parallelizzazione

Nel problema di explainability in cui si richiedono numerose esecuzioni
dell'algoritmo genetico, si potrebbe pensare di partizionare il dataset in $m$
parti e adibire $n$ core a ciascuna partizione di modo da eseguire l'algoritmo
genetico parallelo su $n$ core e su $m$ punti contemporaneamente. Ovviamente
è necessaria un'analisi preliminare in cui si cerca di approssimare il numero
di worker più adatto ad una singola esecuzione dell'algoritmo genetico. Il
numero di partizioni del dataset sarà semplicemente calcolato come:

$$\frac{ct}{w}$$

dove $ct$ è il numero di core totali presenti sulla macchina o che si intende
utilizzare e $w$ è il numero di worker stimato al passo precedente.
